##### Copyright 2020 The TensorFlow Quantum Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# VQT in TFQ

Author : Antonio J. Martinez

Contributors : Guillaume Verdon

Created : 2020-Feb-06

Last updated : 2020-Mar-06

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tensorflow/quantum/blob/research/vqt_qmhl/vqt_qmhl.ipynb)

In this notebook, you will explore the combination of quantum computing and classical energy-based models with TensorFlow Quantum.  The system under study is the [2D Heisenberg model](https://en.wikipedia.org/wiki/Heisenberg_model_(quantum)).  You will apply the Variational Quantum Thermalizer (VQT) to produce approximate thermal states of this model.  VQT was first proposed in the paper [here.](https://arxiv.org/abs/1910.02071)

## Install and import dependencies

In [0]:
!pip install --upgrade tensorflow==2.4.1

In [0]:
!pip install tensorflow-quantum

In [0]:
%%capture
import cirq
import itertools
import numpy as np
import random
from scipy import linalg
import seaborn
import sympy
import tensorflow as tf
import tensorflow_probability as tfp
import tensorflow_quantum as tfq

# visualization tools
%matplotlib inline
import matplotlib.pyplot as plt
from cirq.contrib.svg import SVGCircuit

## 2D Heisenberg model

### Hamiltonian definition

This Hamiltonian is supported on a rectangular lattice of qubits:
$$\hat{H}_{\text{heis}} = \sum_{\langle ij\rangle_h} J_{h} \hat{S}_i \cdot \hat{S}_j +  \sum_{\langle ij\rangle_v} J_{v} \hat{S}_i \cdot \hat{S}_j,$$
where $h$ ($v$) denote horizontal (vertical) bonds, while $\langle \cdot \rangle $ represent nearest-neighbor pairings.
  You can build this Hamiltonian using Cirq `PauliString` and `PauliSum` objects:

In [0]:
def get_qubit_grid(rows, cols):
  """Rectangle of qubits returned as a nested list."""
  qubits = []
  for r in range(rows):
    qubits.append([])
    for c in range(cols):
      qubits[-1].append(cirq.GridQubit(r, c))
  return qubits

def get_bond(q0, q1):
  """Given two Cirq qubits, return the PauliSum that bonds them."""
  return cirq.PauliSum.from_pauli_strings([
    cirq.PauliString(cirq.X(q0), cirq.X(q1)),
    cirq.PauliString(cirq.Y(q0), cirq.Y(q1)),
    cirq.PauliString(cirq.Z(q0), cirq.Z(q1))])

def get_heisenberg_hamiltonian(qubits, jh, jv):
  """Returns the 2D Heisenberg Hamiltonian over the given grid of qubits."""
  heisenberg = cirq.PauliSum()
  # Apply horizontal bonds
  for r in qubits:
    for q0, q1 in zip(r, r[1::]):
      heisenberg += jh * get_bond(q0, q1)
  # Apply vertical bonds
  for r0, r1 in zip(qubits, qubits[1::]):
    for q0, q1 in zip(r0, r1):
      heisenberg += jv * get_bond(q0, q1)
  return heisenberg

For visualization and verification purposes, the following function recovers an explicit matrix from a Cirq `PauliSum` given a linear ordering of the qubits which support it:

In [0]:
def pauli_sum_to_matrix(qubits, pauli_sum):
  """Unpacks each pauli string in the pauli sum into a matrix and sums them."""
  matrix = np.zeros((2**len(qubits), 2**len(qubits)), dtype=np.complex128)
  for pauli_string in pauli_sum:
    coeff = pauli_string.coefficient
    bare_string = pauli_string/coeff
    matrix += coeff*bare_string.dense(qubits)._unitary_()
  return matrix

### Target density matrix

Here you define the parameters of the system to be learned.  The 2D Heisenberg model is defined by the number of rows and columns in the qubit lattice, the bond strengths in the horizontal and vertical directions, and the inverse temperature $\beta$.  Here, we use the same parameters as in the associated paper:

In [0]:
num_rows = 2
num_cols = 2
jh = 1
jv = 0.6
beta = 2.6

# Get the grid of qubits.
all_qubits = get_qubit_grid(num_rows, num_cols)
all_qubits_flat = [q for r in all_qubits for q in r]

Given a Hamiltonian $\hat{H}$ and an inverse temperature $\beta$, the thermal state $\rho_T$ is given by
$$\rho_T = e^{-\beta \hat{H}}.$$
Since our target system is small, you can compute this matrix exponential directly, using the `PauliSum`-to-matrix converter defined above:

In [0]:
num_H = pauli_sum_to_matrix(
    all_qubits_flat, get_heisenberg_hamiltonian(all_qubits, jh, jv))
heisenberg_exp = linalg.expm(-beta*num_H)
exact_thermal_state = np.true_divide(heisenberg_exp, np.trace(heisenberg_exp))
seaborn.heatmap(abs(exact_thermal_state))

Recall that any density matrix $\rho$ [can be written as](https://en.wikipedia.org/wiki/Density_matrix#Definition)
$$\rho = \sum_i p_i |\psi_i\rangle\langle\psi_i|,$$
where $|\psi_i\rangle$ is a pure state and $p_i$ is the classical probability of encoutering that state in the mixture.   Since TFQ is a pure state simulator, we will emulate density matrices by outputting pure states according to their probabilities $p_i$, which by the equation above is equivalent to outputting the full density matrix.  We define here a function that converts such a list of pure states into the associated density matrix:

In [0]:
def pure_state_list_to_density_matrix(pure_states):
  """Return the uniform mixture of the given list of pure states."""
  dim = len(pure_states[0].numpy())
  n_s = pure_states.shape[0]
  thermal_state = np.zeros((dim, dim), dtype=np.complex128)
  for i in range(n_s):
    psi = pure_states[i].numpy()
    thermal_state += np.outer(psi, psi)
  return np.true_divide(thermal_state, n_s)

Finally, to track the performance of our models, we need a measure of the distance of our estimated density matrix $\tilde{\rho}$ from the target density matrix $\rho_T$.  One common metric is the [fidelity](https://en.wikipedia.org/wiki/Fidelity_of_quantum_states), which is defined as
$$F(\tilde{\rho}, \rho_T) = \text{tr}\left[\sqrt{\sqrt{\tilde{\rho}}\rho_T\sqrt{\tilde{\rho}}}\right]^2.$$
This is tractable to compute because our model system is small.  Below we define a function that computes this quantity:

In [0]:
def fidelity(dm1, dm2):
  """Calculate the fidelity between the two given density matrices."""
  dm1_sqrt = linalg.sqrtm(dm1)
  return abs(np.trace(linalg.sqrtm(
      np.matmul(dm1_sqrt, np.matmul(dm2, dm1_sqrt))))) ** 2

## Energy based models

Energy based models are a type of machine learning ansatze inspired by physics and exponential families.  The advantage of using energy based models for probabilistic modeling is that fair samples can be drawn from the distributions they define without requiring computation of their partition functions.

One specific class of EBM is the Boltzmann machine.  The energy of a spin configuration $x \in \{-1, 1\}^n$ in this model is defined as:
$$E(x) = -\sum_{i, j}w_{ij} x_i x_j - \sum_i b_i x_i.$$

This classical model can be easily converted into a quantum mechanical Ising model by replacing each bit with the Pauli $Z$ operator, and considering the usual mapping of the spin to qubit pictures 1 -> $|0\rangle$ and $-1$ -> $|1\rangle$.

In the special case where the connection weights $w_{ij}$ are all zero, the Boltzmann machine is reduced to a product of independent Bernoulli distributions over the set of qubits.  This "Bernoulli EBM" has many simplifying properties, and hence you will explore this EBM first in the examples below.  Later in the notebook, you will apply the full Boltzmann EBM to VQT.

###  Energy functions

Here we define functions which compute the energy of a Boltzmann or Bernoulli EBM given the weight, biases, and bitstrings:

In [0]:
def bitstring_to_spin_config(bitstring):
  """Implements the mapping from the qubit to the spin picture."""
  return [-1 if b == 1 else 1 for b in bitstring]

def spin_config_to_bitstring(spin_config):
  """Implements the mapping from the spin to the qubit picture."""
  return [0 if s == 1 else 1 for s in spin_config]

def ebm_energy(spin_config, biases, weights=None):
  """Given a rank-2 tensor representing the weight matrix and a rank-1 tensor
  representing the biases, calculate the energy of the spin configuration."""
  energy = 0
  if weights is not None:
    for w_row, xi in zip(weights.numpy(), spin_config):
      for wij, xj in zip(w_row, spin_config):
        energy -= wij*xi*xj
  for bi, xi in zip(biases.numpy(), spin_config):
    energy -= bi*xi
  return energy

def ebm_energy_avg(spin_config_list, biases, weights=None):
  """Average energy over a set of spin configuration samples."""
  energy_avg = 0
  for spin_config in spin_config_list:
    energy_avg += ebm_energy(spin_config, biases, weights)
  energy_avg /= len(spin_config_list)
  return energy_avg

We also define functions which initialize TF Variables for our weights and biases.  Initializing all weights and biases near 0 means we begin near the uniform distribution, which can also be thought of as starting with a high temperature prior:

In [0]:
def get_initialized_ebm_biases(num_units):
  return tf.Variable(
      tf.random.uniform(minval=-0.1, maxval=0.1, shape=[num_units],
                        dtype=tf.float32), dtype=tf.float32)
def get_initialized_ebm_weights(num_units):
  return tf.Variable(
      tf.random.uniform(minval=-0.1, maxval=0.1,
                        shape=[num_units, num_units],dtype=tf.float32), dtype=tf.float32)

### EBM derivatives

The derivative of an EBM given a bitstring is easy to compute.  In fact, the derivatives are independent of the weights and biases:
$$\nabla_{w_{ij}}E(x) = -x_ix_j\quad \text{and}\quad \nabla_{b_{i}}E(x) = -x_i.$$
Information about the weights and biases enters by averaging these derivates over samples from the EBM.

In [0]:
def ebm_weights_derivative(spin_config):
  w_deriv = np.zeros((len(spin_config), len(spin_config)))
  for i, x_i in enumerate(spin_config):
    for j, x_j in enumerate(spin_config):
      w_deriv[i][j] = -x_i*x_j
  return w_deriv

def ebm_biases_derivative(spin_config):
  b_deriv = np.zeros(len(spin_config))
  for i, x_i in enumerate(spin_config):
    b_deriv[i] = -x_i
  return b_deriv

def ebm_weights_derivative_avg(spin_config_list):
  w_deriv = np.zeros((len(spin_config_list[0]), len(spin_config_list[0])))
  for spin_config in spin_config_list:
    w_deriv += ebm_weights_derivative(spin_config)
  return np.true_divide(w_deriv, len(spin_config_list))

def ebm_biases_derivative_avg(spin_config_list):
  b_deriv = np.zeros(len(spin_config_list[0]))
  for spin_config in spin_config_list: 
    b_deriv += ebm_biases_derivative(spin_config)
  return np.true_divide(b_deriv, len(spin_config_list))














### Classical VQT loss gradients

As discussed in the paper, the gradient of the VQT loss function can be calculated without computing entropies or partition functions.  For example, the gradient of the VQT free energy loss with respect to the classical model parameters can be writtent as:
$$\partial_{\theta} \mathcal{L}_{\text{fe}} =\mathbb{E}_{x\sim p_{\theta}(x)}[(E_{\theta}(x)-\beta H_{\phi}(x) ) \nabla_{\theta}E_{\theta}(x) ]-(\mathbb{E}_{x\sim p_{\theta}(x)}[E_{\theta}(x)-\beta H_{\phi}(x)]) ( \mathbb{E}_{y\sim p_{\theta}(y)}[\nabla_{\theta}E_{\theta}(y)] ).$$
Below these gradients are defined for the general Boltzmann EBM.  In the VQT gradients, each entry in `bitstring_list` corresponds to the entry with the same index in `energy_losses`, where for each bitstring $x$, we compute the product $\beta\langle x|H|x\rangle$.  The list of bitstrings is assumed to be sampled from the EBM.


In [0]:
def get_vqt_weighted_weights_grad_product(
    energy_losses, spin_config_list, biases, weights):
  """Implements the first term in the derivative of the FE loss,
  for the weights of a Boltzmann EBM."""
  w_deriv = np.zeros((len(spin_config_list[0]), len(spin_config_list[0])))
  for e_loss, spin_config in zip(energy_losses, spin_config_list):
    w_deriv = w_deriv + (
        ebm_energy(spin_config, biases, weights) - e_loss 
        )*ebm_weights_derivative(spin_config)
  return np.true_divide(w_deriv, len(energy_losses))

def get_vqt_weighted_biases_grad_product(
    energy_losses, spin_config_list, biases, weights=None):
  """Implements the first term in the derivative of the FE loss,
  for the biases of a Boltzmann EBM."""
  b_deriv = np.zeros(len(spin_config_list[0]))
  for e_loss, spin_config in zip(energy_losses, spin_config_list):
    b_deriv = b_deriv + (
        ebm_energy(spin_config, biases, weights) - e_loss
        )*ebm_biases_derivative(spin_config)
  return np.true_divide(b_deriv, len(energy_losses))

def get_vqt_factored_weights_grad_product(
    energy_losses, spin_config_list, biases, weights):
  """Implements the second term in the derivative of the FE loss,
  for the weights of a Boltzmann EBM."""
  energy_losses_avg = tf.reduce_mean(energy_losses)
  classical_energy_avg = ebm_energy_avg(spin_config_list, biases, weights)
  energy_diff_avg = classical_energy_avg - energy_losses_avg
  return energy_diff_avg*ebm_weights_derivative_avg(spin_config_list)

def get_vqt_factored_biases_grad_product(
    energy_losses, spin_config_list, biases, weights=None):
  """Implements the second term in the derivative of the FE loss,
  for the biases of a Boltzmann EBM."""
  energy_losses_avg = tf.reduce_mean(energy_losses)
  classical_energy_avg = ebm_energy_avg(spin_config_list, biases, weights)
  energy_diff_avg = classical_energy_avg - energy_losses_avg
  return energy_diff_avg*ebm_biases_derivative_avg(spin_config_list)

## Model components

### Ansatz unitary

The parameterized unitary ansatz you will use consists of alternating layers of general single qubit rotations and nearest-neighbor entangling gates:

In [0]:
def get_rotation_1q(q, a, b, c):
  """General single qubit rotation."""
  return cirq.Circuit(cirq.X(q) ** a, cirq.Y(q) ** b, cirq.Z(q) ** c)

def get_rotation_2q(q0, q1, a):
  """Exponent of entangling CNOT gate."""
  return cirq.Circuit(cirq.CNotPowGate(exponent=a)(q0, q1))

def get_layer_1q(qubits, layer_num, name):
  """Apply single qubit rotations to all the given qubits."""
  layer_symbols = []
  circuit = cirq.Circuit()
  for n, q in enumerate(qubits):
    a, b, c = sympy.symbols(
        "a{2}_{0}_{1} b{2}_{0}_{1} c{2}_{0}_{1}".format(layer_num, n, name))
    layer_symbols += [a, b, c]
    circuit += get_rotation_1q(q, a, b, c)
  return circuit, layer_symbols

def get_layer_2q(qubits, layer_num, name):
  """Apply CNOT gates to all pairs of nearest-neighbor qubits."""
  layer_symbols = []
  circuit = cirq.Circuit()
  for n, (q0, q1) in enumerate(zip(qubits[::2], qubits[1::2])):
    a = sympy.symbols("a{2}_{0}_{1}".format(layer_num, n, name))
    layer_symbols += [a]
    circuit += get_rotation_2q(q0, q1, a)
  shifted_qubits = qubits[1::]+[qubits[0]]
  for n, (q0, q1) in enumerate(zip(shifted_qubits[::2], shifted_qubits[1::2])):
    a = sympy.symbols("a{2}_{0}_{1}".format(layer_num, n+1, name))
    layer_symbols += [a]
    circuit += get_rotation_2q(q0, q1, a)
  return circuit, layer_symbols

def get_one_full_layer(qubits, layer_num, name):
  """Stack the one- and two-qubit parameterized circuits."""
  circuit = cirq.Circuit()
  all_symbols = []
  new_circ, new_symb = get_layer_1q(qubits, layer_num, name)
  circuit += new_circ
  all_symbols += new_symb
  new_circ, new_symb = get_layer_2q(qubits, layer_num + 1, name)
  circuit += new_circ
  all_symbols += new_symb
  return circuit, all_symbols

def get_model_unitary(qubits, num_layers, name=""):
  """Build our full parameterized model unitary."""
  circuit = cirq.Circuit()
  all_symbols = []
  for i in range(num_layers):
    new_circ, new_symb = get_one_full_layer(qubits, 2*i, name)
    circuit += new_circ
    all_symbols += new_symb
  return circuit, all_symbols

### Bitstring injector

You also need a way to feed bitstrings into the quantum model.  These bitstrings can be prepared by applying an X gate to every qubit that should be excited.  The following function returns a parameterized circuit which prepares any given bitstring:

In [0]:
def get_bitstring_circuit(qubits):
  """Returns wall of parameterized X gates and the bits used to turn them on."""
  circuit = cirq.Circuit()
  all_symbols = []
  for n, q in enumerate(qubits):
    new_bit = sympy.Symbol("bit_{}".format(n))
    circuit += cirq.X(q) ** new_bit
    all_symbols.append(new_bit)
  return circuit, all_symbols

## Factorized latent state

### Bernoulli EBM

The Bernoulli EBM can be used to parameterize a factorized latent state.  The probability of sampling a 1 from a unit with bias $b$ is:

$$p = \frac{e^b}{e^b + e^{-b}}$$

Since the units of a Bernoulli EBM are independent, the probability of a given spin configuration is simply the product of the individual unit probabilities:

$$p(x) = \prod_i\frac{e^{x_ib_i}}{e^{b_i} + e^{-b_i}}$$

This distribution is easy to sample from.

In [0]:
def bernoulli_spin_p1(b):
  return np.exp(b)/(np.exp(b) + np.exp(-b))

def sample_spins_bernoulli(num_samples, biases):
  prob_list = []
  for bias in biases.numpy():
    prob_list.append(bernoulli_spin_p1(bias))
  # The `probs` keyword specifies the probability of a 1 event
  latent_dist = tfp.distributions.Bernoulli(probs=prob_list, dtype=tf.float32)
  bit_samples = latent_dist.sample(num_samples).numpy()
  spin_samples = []
  for sample in bit_samples:
    spin_samples.append([])
    for bit in sample:
      if bit == 0:
        spin_samples[-1].append(-1)
      else:
        spin_samples[-1].append(1)
  return spin_samples

The entropy of a single unit with bias $b$ in our Bernoulli EBM is:

$S = \frac{be^{b} - be^{-b}}{e^{b} + e^{-b}}- \log[e^{b} + e^{-b}]$

For a factorized latent distribution, the entropy is simply the sum of the entropies of the individual factors.

In [0]:
def bernoulli_factor_partition(b):
  return np.exp(b) + np.exp(-b)

def bernoulli_partition(biases):
  partition = 1
  for bias in biases.numpy():
    partition *= bernoulli_factor_partition(bias)
  return partition

def bernoulli_factor_entropy(b):
  Z = bernoulli_factor_partition(b)
  return (b*np.exp(b) - b*np.exp(-b))/Z - np.log(Z)

def bernoulli_entropy(biases):
  entropy = 0
  for bias in biases.numpy():
    entropy += bernoulli_factor_entropy(bias)
  return entropy

Finally we define a function for converting the classical Bernoulli distribution into an Ising model whose expectation values can be simulated in the TFQ ops:

In [0]:
def bernoulli_ebm_to_ising(qubits, biases, bare=False):
  pauli_s_list = []
  for i, bi in enumerate(biases.numpy()):
    if bare:
      coeff = 1.0
    else:
      coeff = bi
    pauli_s_list.append(cirq.PauliString(coeff, cirq.Z(qubits[i])))
  if bare:
    return pauli_s_list
  return cirq.PauliSum.from_pauli_strings(pauli_s_list)

### VQT

Build and view our unitary model and set up the TFQ Expectation Op inputs:

In [0]:
# Number of bitstring samples from our classical model to average over
num_samples = 300

# Number of rotations-plus-entanglement layers to stack.
# Note that the depth required to reach a given fidelity increases depending on
# the temperature and Hamiltonian parameters.
num_layers = 4

# Build the model unitary and visible state circuits
U, model_symbols = get_model_unitary(all_qubits_flat, num_layers)
V, bit_symbols = get_bitstring_circuit(all_qubits_flat)
visible_state = tfq.convert_to_tensor([V + U])

# Make a copy of the visible state for each bitstring we will sample
tiled_visible_state = tf.tile(visible_state, [num_samples])

# Upconvert symbols to tensors
vqt_symbol_names = tf.identity(tf.convert_to_tensor(
    [str(s) for s in bit_symbols + model_symbols], dtype=tf.dtypes.string))

# Build and tile the Hamiltonian
H = get_heisenberg_hamiltonian(all_qubits, jh, jv)
tiled_H = tf.tile(tfq.convert_to_tensor([[H]]), [num_samples, 1])

# Get the expectation op with a differentiator attached
expectation = tfq.differentiators.ForwardDifference().generate_differentiable_op(
            analytic_op=tfq.get_expectation_op())

SVGCircuit(U)

We can use gradient descent on the model parameters thanks to TFQ, and our classical model parameters are tractable due to our use of an energy based model.  The factorized nature of our latent space also allows us to efficiently obtain a loss function.

In [0]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.03)

# Initialize our model variables
vqt_model_params = tf.Variable(
    tf.random.uniform(minval=-0.1, maxval=0.1,
                      shape=[len(model_symbols)], dtype=tf.float32),
    dtype=tf.float32)

# Keep track of metrics during training
vqt_loss_history = []
vqt_fidelity_history = []
vqt_model_params_history = []
vqt_bias_history = []
vqt_density_matrix_history = []

# Initialize our EBM variables
vqt_biases = get_initialized_ebm_biases(len(all_qubits_flat))

# The innermost training step, where gradients are taken and applied
def vqt_train_step():
  # Sample from our EBM
  spin_config_list = sample_spins_bernoulli(num_samples, vqt_biases)
  bitstring_list = [spin_config_to_bitstring(s) for s in spin_config_list]
  bitstring_tensor = tf.convert_to_tensor(bitstring_list, dtype=tf.float32)

  # Use the samples to find gradient of the loss w.r.t. model parameters.
  with tf.GradientTape() as tape:
    tiled_vqt_model_params = tf.tile([vqt_model_params], [num_samples, 1])
    sampled_expectations = expectation(
        tiled_visible_state,
        vqt_symbol_names,
        tf.concat([bitstring_tensor, tiled_vqt_model_params], 1),
        tiled_H)
    energy_losses = beta*sampled_expectations
    energy_losses_avg = tf.reduce_mean(energy_losses)
  vqt_model_gradients = tape.gradient(energy_losses_avg, [vqt_model_params])

  # Build the classical model gradients
  weighted_biases_grad = get_vqt_weighted_biases_grad_product(
    energy_losses, spin_config_list, vqt_biases)
  factored_biases_grad = get_vqt_factored_biases_grad_product(
    energy_losses, spin_config_list, vqt_biases)
  biases_grad = tf.subtract(weighted_biases_grad, factored_biases_grad)

  # Apply the gradients
  optimizer.apply_gradients(zip([vqt_model_gradients[0], biases_grad],
                                [vqt_model_params, vqt_biases]))

  # Sample pure states to build the current estimate of the density matrix
  many_states = tfq.layers.State()(
      tiled_visible_state,
      symbol_names=vqt_symbol_names,
      symbol_values=tf.concat([bitstring_tensor, tiled_vqt_model_params], 1)
  )
  vqt_density_matrix_history.append(pure_state_list_to_density_matrix(many_states))

  # Record the history
  vqt_loss_history.append((energy_losses_avg - bernoulli_entropy(vqt_biases)).numpy())
  vqt_fidelity_history.append(
      fidelity(vqt_density_matrix_history[-1], exact_thermal_state))
  vqt_model_params_history.append(vqt_model_params.numpy())
  vqt_bias_history.append(vqt_biases.numpy())
  
  print("Current loss:")
  print(vqt_loss_history[-1])
  print("Current fidelity to optimal state:")
  print(vqt_fidelity_history[-1])
  print("Current estimated density matrix:")
  plt.figure()
  seaborn.heatmap(abs(vqt_density_matrix_history[-1]))
  plt.show()

With setup complete, we can now optimize our Heisenberg VQT.

In [0]:
def vqt_train(epochs):
  for epoch in range(epochs):
    vqt_train_step()
    print ('Epoch {} finished'.format(epoch + 1))

vqt_train(100)

We plot our metrics and visualize the motion of the parameters during training:

In [0]:
plt.plot(vqt_loss_history)
plt.xlabel('Epoch #')
plt.ylabel('Loss [free energy]')

In [0]:
plt.plot(vqt_fidelity_history)
plt.xlabel('Epoch #')
plt.ylabel('Fidelity with exact state')

## Classically correlated latent state

### Boltzmann machine EBM

The Bernoulli distribution is only able to inject entropy into our density matrix.  To encode classical correlations, we need to move beyond a factorized latent state.  This can be accomplished by allowing the weights of our Boltzmann machine to be non-zero.

Now that there are correlations, sampling from the model becomes non-trivial.  The probability of bitstring $x$ is:

$P(x) = \frac{\exp(-E(x))}{\sum_{y\in\{-1, 1\}^n} \exp(-E(y))}$

In general this function is intractable to compute directly; however, we can still obtain samples from the distribution efficiently. Markov chain Monte Carlo (MCMC) is one family of procedures for this efficient sampling.  Here, we use the simplest example of MCMC, the [Metropolis-Hastings](https://en.wikipedia.org/wiki/Metropolis%E2%80%93Hastings_algorithm) algorithm:


In [0]:
def make_proposal(y):
  """Flip spins in y to generate a new sample."""
  coin = tfp.distributions.Bernoulli(probs=[0.75]*len(y))
  samples = coin.sample(1).numpy()[0]
  x = []
  for s_i, y_i in zip(samples, y):
    if s_i:
      x.append(y_i)
    else:
      if y_i == 1:
        x.append(-1)
      else:
        x.append(1)
  return x

def sample_boltzmann(burn_in, num_samples, skip, initial_state, biases, weights):
  """Walk towards and sample from regions of high probability."""
  current_state = initial_state
  all_samples = []
  for i in range(burn_in + skip*num_samples):
    proposal = make_proposal(current_state)
    proposal_energy = ebm_energy(proposal, biases, weights)
    current_energy = ebm_energy(current_state, biases, weights)
    acceptance = min(np.exp(-proposal_energy)/np.exp(-current_energy), 1)
    threshold = random.random()
    if threshold <= acceptance:
      current_state = proposal
    if i >= burn_in:
      if (i - burn_in)%skip == 0:
        all_samples.append(current_state)
  return all_samples

Since there are now correlations between the bits, the partition function and entropy can no longer be computed in a scalable way.  However, for the small system size considered here, these quantities can be illustrative.

In [0]:
def boltzmann_partition(biases, weights):
  partition_value = 0
  for spin_config in itertools.product([-1, 1], repeat=biases.shape[0]):
    partition_value += np.exp(-ebm_energy(spin_config, biases, weights))
  return partition_value

def boltzmann_entropy(biases, weights):
  Z = boltzmann_partition(biases, weights)
  Z_log = np.log(Z)
  unnormalized = 0
  for spin_config in itertools.product([-1, 1], repeat=biases.shape[0]):
    this_energy = ebm_energy(spin_config, biases, weights)
    unnormalized += np.exp(-this_energy)*(-this_energy - Z_log)
  return -unnormalized/Z

Finally we define a function for converting the classical Boltzmann machine into an Ising model whose expectation values can be simulated in the TFQ ops:

In [0]:
def boltzmann_ebm_to_ising(qubits, biases, weights, bare=False):
  pauli_s_list = []
  for i, w_row in enumerate(weights.numpy()):
    for j, wij in enumerate(w_row):
      init_list = [cirq.Z(q) for qi, q in enumerate(qubits) if qi == i or qi == j]
      if bare:
        coeff = 1.0
      else:
        coeff = wij
      pauli_s_list.append(cirq.PauliString(coeff, init_list))
  for i, bi in enumerate(biases.numpy()):
    if bare:
      coeff = 1.0
    else:
      coeff = bi
    pauli_s_list.append(cirq.PauliString(coeff, cirq.Z(qubits[i])))
  if bare:
    return pauli_s_list
  return cirq.PauliSum.from_pauli_strings(pauli_s_list)

### VQT

In [0]:
# Initialize our model variables
vqt_model_params = tf.Variable(
    tf.random.uniform(minval=-0.1, maxval=0.1, shape=[len(model_symbols)],
                      dtype=tf.float32), dtype=tf.float32)

# Define the learning hyperparameters
burn_in = 100
skip = 7
optimizer = tf.keras.optimizers.Adam(learning_rate=0.025)

# Keep track of metrics during training
vqt_loss_history = []
vqt_fidelity_history = []
vqt_model_params_history = []
vqt_weights_history = []
vqt_bias_history = []
vqt_density_matrix_history = []

# Initialize our EBM variables
vqt_weights = get_initialized_ebm_weights(len(all_qubits_flat))
vqt_biases = get_initialized_ebm_biases(len(all_qubits_flat))

# The innermost training step, where gradients are taken and applied
def vqt_train_step():
  # Sample from our EBM
  spin_config_list = sample_boltzmann(burn_in, num_samples, skip,
                                      vqt_train_step.initial_state,
                                      vqt_biases, vqt_weights)
  vqt_train_step.initial_state = spin_config_list[-1]
  bitstring_list = [spin_config_to_bitstring(s) for s in spin_config_list]
  bitstring_tensor = tf.convert_to_tensor(bitstring_list, dtype=tf.float32)

  # Use the samples to find gradient of the loss w.r.t. model parameters.
  with tf.GradientTape() as tape:
    tiled_vqt_model_params = tf.tile([vqt_model_params], [num_samples, 1])
    sampled_expectations = expectation(
        tiled_visible_state,
        vqt_symbol_names,
        tf.concat([bitstring_tensor, tiled_vqt_model_params], 1),
        tiled_H)
    energy_losses = beta*sampled_expectations
    energy_losses_avg = tf.reduce_mean(energy_losses)
  vqt_model_gradients = tape.gradient(energy_losses_avg, [vqt_model_params])

  # Build the classical model gradients
  weighted_biases_grad = get_vqt_weighted_biases_grad_product(
    energy_losses, spin_config_list, vqt_biases, vqt_weights)
  factored_biases_grad = get_vqt_factored_biases_grad_product(
    energy_losses, spin_config_list, vqt_biases, vqt_weights)
  biases_grad = tf.subtract(weighted_biases_grad, factored_biases_grad)
  weighted_weights_grad = get_vqt_weighted_weights_grad_product(
    energy_losses, spin_config_list, vqt_weights, vqt_weights)
  factored_weights_grad = get_vqt_factored_weights_grad_product(
    energy_losses, spin_config_list, vqt_weights, vqt_weights)
  weights_grad = tf.subtract(weighted_weights_grad, factored_weights_grad)

  # Apply the gradients
  optimizer.apply_gradients(
      zip([vqt_model_gradients[0], weights_grad, biases_grad],
          [vqt_model_params, vqt_weights, vqt_biases]))

  # Sample pure states to build the current estimate of the density matrix
  many_states = tfq.layers.State()(
      tiled_visible_state,
      symbol_names=vqt_symbol_names,
      symbol_values=tf.concat([bitstring_tensor, tiled_vqt_model_params], 1)
  )
  vqt_density_matrix_history.append(pure_state_list_to_density_matrix(many_states))

  # Record the history
  vqt_loss_history.append(
      (energy_losses_avg - boltzmann_entropy(vqt_biases, vqt_weights)).numpy())
  vqt_fidelity_history.append(
      fidelity(vqt_density_matrix_history[-1], exact_thermal_state))
  vqt_model_params_history.append(vqt_model_params.numpy())
  vqt_weights_history.append(vqt_weights.numpy())
  vqt_bias_history.append(vqt_biases.numpy())
  
  print("Current loss:")
  print(vqt_loss_history[-1])
  print("Current fidelity to optimal state:")
  print(vqt_fidelity_history[-1])
  print("Current estimated density matrix:")
  plt.figure()
  seaborn.heatmap(abs(vqt_density_matrix_history[-1]))
  plt.show()

vqt_train_step.initial_state = [1]*len(bit_symbols)

In [0]:
def vqt_train(epochs):
  for epoch in range(epochs):
    vqt_train_step()
    print ('Epoch {} finished'.format(epoch + 1))

vqt_train(100)

In [0]:
plt.plot(vqt_loss_history)
plt.xlabel('Epoch #')
plt.ylabel('Loss [free energy]')

In [0]:
plt.plot(vqt_fidelity_history)
plt.xlabel('Epoch #')
plt.ylabel('Fidelity with exact state')